In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76700: Some(http://gw02.itversity.com:4054)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76700

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val sales_path = hdfs_home +"/dataSets/spark-guide/retail-data/by-day/*.csv"
val fakeIntDF_path = hdfs_home +"/dataSets/spark-guide/simple-ml-integers"
var simpleDF_path = hdfs_home +"/dataSets/spark-guide/simple-ml"
val scaleDF_path = hdfs_home +"/dataSets/spark-guide/simple-ml-scaling"

sales_path = /user/kranthidr/dataSets/spark-guide/retail-data/by-day/*.csv
fakeIntDF_path = /user/kranthidr/dataSets/spark-guide/simple-ml-integers
simpleDF_path = /user/kranthidr/dataSets/spark-guide/simple-ml
scaleDF_path = /user/kranthidr/dataSets/spark-guide/simple-ml-scaling


/user/kranthidr/dataSets/spark-guide/simple-ml-scaling

In [6]:
// in Scala
val sales = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(sales_path)
  .coalesce(5)
  .where("Description IS NOT NULL")

sales = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [7]:
val fakeIntDF = spark.read.parquet(fakeIntDF_path)
var simpleDF = spark.read.json(simpleDF_path)
val scaleDF = spark.read.parquet(scaleDF_path)

fakeIntDF = [int1: int, int2: int ... 1 more field]
simpleDF = [color: string, lab: string ... 2 more fields]
scaleDF = [id: int, features: vector]


[id: int, features: vector]

In [8]:
// COMMAND ----------

sales.cache()
sales.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [9]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.Tokenizer
val tkn = new Tokenizer().setInputCol("Description")
tkn.transform(sales.select("Description")).show(false)

+-----------------------------------+------------------------------------------+
|Description                        |tok_c0191bb3f9eb__output                  |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, , water, transfer, tattoos]      |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

tkn = tok_c0191bb3f9eb


tok_c0191bb3f9eb

In [10]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.StandardScaler
val ss = new StandardScaler().setInputCol("features")
ss.fit(scaleDF).transform(scaleDF).show(false)

+---+--------------+------------------------------------------------------------+
|id |features      |stdScal_cc05555257c9__output                                |
+---+--------------+------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
|1  |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968]   |
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
|1  |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968]   |
|1  |[3.0,10.1,3.0]|[3.5856858280031805,2.3609991401715313,1.7928429140015902]  |
+---+--------------+------------------------------------------------------------+



ss = stdScal_cc05555257c9


stdScal_cc05555257c9

In [11]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.RFormula
val supervised = new RFormula()
  .setFormula("lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

+-----+----+------+------------------+--------------------+-----+
|color| lab|value1|            value2|            features|label|
+-----+----+------+------------------+--------------------+-----+
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|
| blue| bad|    12|14.386294994851129|(10,[2,3,6,9],[12...|  0.0|
|green|good|    15| 38.97187133755819|(10,[1,2,3,5,8],[...|  1.0|
|green|good|    12|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
|green| bad|    16|14.386294994851129|(10,[1,2,3,5,8],[...|  0.0|
|  red|good|    35|14.386294994851129|(10,[0,2,3,4,7],[...|  1.0|
|  red| bad|     1| 38.97187133755819|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|     2|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|    16|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red|good|    45| 38.97187133755819|(10,[0,2,3,4,7],[...|  1.0|
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| ba

supervised = RFormula(lab ~ . + color:value1 + color:value2) (uid=rFormula_d970cd13cc35)


RFormula(lab ~ . + color:value1 + color:value2) (uid=rFormula_d970cd13cc35)

In [12]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.SQLTransformer

val basicTransformation = new SQLTransformer()
  .setStatement("""
    SELECT sum(Quantity), count(*), CustomerID
    FROM __THIS__
    GROUP BY CustomerID
  """)

basicTransformation.transform(sales).show()

+-------------+--------+----------+
|sum(Quantity)|count(1)|CustomerID|
+-------------+--------+----------+
|          854|     117|   17884.0|
|          541|      27|   14285.0|
|         1140|      30|   13918.0|
|         1542|      30|   13094.0|
|          204|      76|   13533.0|
|          109|      11|   13973.0|
|          206|      23|   12493.0|
|          491|     152|   13956.0|
|           88|       7|   14473.0|
|          290|      98|   13607.0|
|           34|       6|   14768.0|
|           97|      12|   16596.0|
|          630|      72|   17633.0|
|          244|      31|   16561.0|
|          493|      64|   16629.0|
|          159|      38|   17267.0|
|          138|      18|   15776.0|
|          150|      16|   14024.0|
|          119|      62|   14452.0|
|          440|     143|   16916.0|
+-------------+--------+----------+
only showing top 20 rows



basicTransformation = sql_16f6206a0cd2


sql_16f6206a0cd2

In [13]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.VectorAssembler
val va = new VectorAssembler().setInputCols(Array("int1", "int2", "int3"))
va.transform(fakeIntDF).show()

+----+----+----+---------------------------------+
|int1|int2|int3|vecAssembler_67b742cb1c97__output|
+----+----+----+---------------------------------+
|   1|   2|   3|                    [1.0,2.0,3.0]|
|   4|   5|   6|                    [4.0,5.0,6.0]|
|   7|   8|   9|                    [7.0,8.0,9.0]|
+----+----+----+---------------------------------+



va = vecAssembler_67b742cb1c97


vecAssembler_67b742cb1c97

In [14]:
// COMMAND ----------

// in Scala
val contDF = spark.range(20).selectExpr("cast(id as double)")

contDF = [id: double]


[id: double]

In [15]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.Bucketizer
val bucketBorders = Array(-1.0, 5.0, 10.0, 250.0, 600.0)
val bucketer = new Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(contDF).show()

+----+-------------------------------+
|  id|bucketizer_5414679ff0ef__output|
+----+-------------------------------+
| 0.0|                            0.0|
| 1.0|                            0.0|
| 2.0|                            0.0|
| 3.0|                            0.0|
| 4.0|                            0.0|
| 5.0|                            1.0|
| 6.0|                            1.0|
| 7.0|                            1.0|
| 8.0|                            1.0|
| 9.0|                            1.0|
|10.0|                            2.0|
|11.0|                            2.0|
|12.0|                            2.0|
|13.0|                            2.0|
|14.0|                            2.0|
|15.0|                            2.0|
|16.0|                            2.0|
|17.0|                            2.0|
|18.0|                            2.0|
|19.0|                            2.0|
+----+-------------------------------+



bucketBorders = Array(-1.0, 5.0, 10.0, 250.0, 600.0)
bucketer = bucketizer_5414679ff0ef


bucketizer_5414679ff0ef

In [16]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.QuantileDiscretizer
val bucketer = new QuantileDiscretizer().setNumBuckets(5).setInputCol("id").setOutputCol("buckets")
val fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

+----+-------+
|  id|buckets|
+----+-------+
| 0.0|    0.0|
| 1.0|    0.0|
| 2.0|    0.0|
| 3.0|    1.0|
| 4.0|    1.0|
| 5.0|    1.0|
| 6.0|    1.0|
| 7.0|    2.0|
| 8.0|    2.0|
| 9.0|    2.0|
|10.0|    2.0|
|11.0|    2.0|
|12.0|    3.0|
|13.0|    3.0|
|14.0|    3.0|
|15.0|    4.0|
|16.0|    4.0|
|17.0|    4.0|
|18.0|    4.0|
|19.0|    4.0|
+----+-------+



bucketer = quantileDiscretizer_a32c132e9617
fittedBucketer = quantileDiscretizer_a32c132e9617


quantileDiscretizer_a32c132e9617

In [17]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.StandardScaler
val sScaler = new StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+----------------------------+
| id|      features|stdScal_146c02cb8067__output|
+---+--------------+----------------------------+
|  0|[1.0,0.1,-1.0]|        [1.19522860933439...|
|  1| [2.0,1.1,1.0]|        [2.39045721866878...|
|  0|[1.0,0.1,-1.0]|        [1.19522860933439...|
|  1| [2.0,1.1,1.0]|        [2.39045721866878...|
|  1|[3.0,10.1,3.0]|        [3.58568582800318...|
+---+--------------+----------------------------+



sScaler = stdScal_146c02cb8067


stdScal_146c02cb8067

In [18]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.MinMaxScaler
val minMax = new MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
val fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

+---+--------------+-------------------------------+
| id|      features|minMaxScal_fccd0a94edc4__output|
+---+--------------+-------------------------------+
|  0|[1.0,0.1,-1.0]|                  [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                  [7.5,5.5,7.5]|
|  0|[1.0,0.1,-1.0]|                  [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                  [7.5,5.5,7.5]|
|  1|[3.0,10.1,3.0]|               [10.0,10.0,10.0]|
+---+--------------+-------------------------------+



minMax = minMaxScal_fccd0a94edc4
fittedminMax = minMaxScal_fccd0a94edc4


minMaxScal_fccd0a94edc4

In [19]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.MaxAbsScaler
val maScaler = new MaxAbsScaler().setInputCol("features")
val fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show()

+---+--------------+-------------------------------+
| id|      features|maxAbsScal_a0affd6200d8__output|
+---+--------------+-------------------------------+
|  0|[1.0,0.1,-1.0]|           [0.33333333333333...|
|  1| [2.0,1.1,1.0]|           [0.66666666666666...|
|  0|[1.0,0.1,-1.0]|           [0.33333333333333...|
|  1| [2.0,1.1,1.0]|           [0.66666666666666...|
|  1|[3.0,10.1,3.0]|                  [1.0,1.0,1.0]|
+---+--------------+-------------------------------+



maScaler = maxAbsScal_a0affd6200d8
fittedmaScaler = maxAbsScal_a0affd6200d8


maxAbsScal_a0affd6200d8

In [20]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.ElementwiseProduct
import org.apache.spark.ml.linalg.Vectors
val scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
val scalingUp = new ElementwiseProduct()
  .setScalingVec(scaleUpVec)
  .setInputCol("features")
scalingUp.transform(scaleDF).show()

+---+--------------+-----------------------------+
| id|      features|elemProd_f9ae05b007b5__output|
+---+--------------+-----------------------------+
|  0|[1.0,0.1,-1.0]|             [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]|             [20.0,16.5,20.0]|
|  0|[1.0,0.1,-1.0]|             [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]|             [20.0,16.5,20.0]|
|  1|[3.0,10.1,3.0]|            [30.0,151.5,60.0]|
+---+--------------+-----------------------------+



scaleUpVec = [10.0,15.0,20.0]
scalingUp = elemProd_f9ae05b007b5


elemProd_f9ae05b007b5

In [21]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.Normalizer
val manhattanDistance = new Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show()

+---+--------------+-------------------------------+
| id|      features|normalizer_a1d94d61198b__output|
+---+--------------+-------------------------------+
|  0|[1.0,0.1,-1.0]|           [0.47619047619047...|
|  1| [2.0,1.1,1.0]|           [0.48780487804878...|
|  0|[1.0,0.1,-1.0]|           [0.47619047619047...|
|  1| [2.0,1.1,1.0]|           [0.48780487804878...|
|  1|[3.0,10.1,3.0]|           [0.18633540372670...|
+---+--------------+-------------------------------+



manhattanDistance = normalizer_a1d94d61198b


normalizer_a1d94d61198b

In [22]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.StringIndexer
val lblIndxr = new StringIndexer().setInputCol("lab").setOutputCol("labelInd")
val idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.show()

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|labelInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|     1.0|
|  red| bad|     1| 38.97187133755819|     0.0|
|  red| bad|     2|14.386294994851129|     0.0|
|  red| bad|    16|14.386294994851129|     0.0|
|  red|good|    45| 38.97187133755819|     1.0|
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|  

lblIndxr = strIdx_5471bf0041ad
idxRes = [color: string, lab: string ... 3 more fields]


[color: string, lab: string ... 3 more fields]

In [23]:
// COMMAND ----------

// in Scala
val valIndexer = new StringIndexer()
  .setInputCol("value1")
  .setOutputCol("valueInd")

valIndexer.fit(simpleDF).transform(simpleDF).show()

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|valueInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     2.0|
| blue| bad|     8|14.386294994851129|     4.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     5.0|
|green|good|    12|14.386294994851129|     0.0|
|green| bad|    16|14.386294994851129|     1.0|
|  red|good|    35|14.386294994851129|     6.0|
|  red| bad|     1| 38.97187133755819|     2.0|
|  red| bad|     2|14.386294994851129|     7.0|
|  red| bad|    16|14.386294994851129|     1.0|
|  red|good|    45| 38.97187133755819|     3.0|
|green|good|     1|14.386294994851129|     2.0|
| blue| bad|     8|14.386294994851129|     4.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     5.0|
|green|good|    12|14.386294994851129|     0.0|
|green| bad|    16|14.386294994851129|     1.0|
|  red|good|    35|14.386294994851129|  

valIndexer = strIdx_1025312f9da9


strIdx_1025312f9da9

In [24]:
// COMMAND ----------

valIndexer.setHandleInvalid("skip")
valIndexer.fit(simpleDF).setHandleInvalid("skip")

strIdx_1025312f9da9

In [25]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.IndexToString
val labelReverse = new IndexToString().setInputCol("labelInd")
labelReverse.transform(idxRes).show()

+-----+----+------+------------------+--------+-----------------------------+
|color| lab|value1|            value2|labelInd|idxToStr_1e939612dd7f__output|
+-----+----+------+------------------+--------+-----------------------------+
|green|good|     1|14.386294994851129|     1.0|                         good|
| blue| bad|     8|14.386294994851129|     0.0|                          bad|
| blue| bad|    12|14.386294994851129|     0.0|                          bad|
|green|good|    15| 38.97187133755819|     1.0|                         good|
|green|good|    12|14.386294994851129|     1.0|                         good|
|green| bad|    16|14.386294994851129|     0.0|                          bad|
|  red|good|    35|14.386294994851129|     1.0|                         good|
|  red| bad|     1| 38.97187133755819|     0.0|                          bad|
|  red| bad|     2|14.386294994851129|     0.0|                          bad|
|  red| bad|    16|14.386294994851129|     0.0|                 

labelReverse = idxToStr_1e939612dd7f


idxToStr_1e939612dd7f

In [26]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.linalg.Vectors
val idxIn = spark.createDataFrame(Seq(
  (Vectors.dense(1, 2, 3),1),
  (Vectors.dense(2, 5, 6),2),
  (Vectors.dense(1, 8, 9),3)
)).toDF("features", "label")
val indxr = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("idxed")
  .setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show

+-------------+-----+-------------+
|     features|label|        idxed|
+-------------+-----+-------------+
|[1.0,2.0,3.0]|    1|[0.0,2.0,3.0]|
|[2.0,5.0,6.0]|    2|[1.0,5.0,6.0]|
|[1.0,8.0,9.0]|    3|[0.0,8.0,9.0]|
+-------------+-----+-------------+



idxIn = [features: vector, label: int]
indxr = vecIdx_397a85d55f52


vecIdx_397a85d55f52

In [27]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder}
val lblIndxr = new StringIndexer().setInputCol("color").setOutputCol("colorInd")
val colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
val ohe = new OneHotEncoder().setInputCol("colorInd")
ohe.transform(colorLab).show()

+-----+--------+---------------------------+
|color|colorInd|oneHot_48b48c72bfc3__output|
+-----+--------+---------------------------+
|green|     1.0|              (2,[1],[1.0])|
| blue|     2.0|                  (2,[],[])|
| blue|     2.0|                  (2,[],[])|
|green|     1.0|              (2,[1],[1.0])|
|green|     1.0|              (2,[1],[1.0])|
|green|     1.0|              (2,[1],[1.0])|
|  red|     0.0|              (2,[0],[1.0])|
|  red|     0.0|              (2,[0],[1.0])|
|  red|     0.0|              (2,[0],[1.0])|
|  red|     0.0|              (2,[0],[1.0])|
|  red|     0.0|              (2,[0],[1.0])|
|green|     1.0|              (2,[1],[1.0])|
| blue|     2.0|                  (2,[],[])|
| blue|     2.0|                  (2,[],[])|
|green|     1.0|              (2,[1],[1.0])|
|green|     1.0|              (2,[1],[1.0])|
|green|     1.0|              (2,[1],[1.0])|
|  red|     0.0|              (2,[0],[1.0])|
|  red|     0.0|              (2,[0],[1.0])|
|  red|   

lblIndxr = strIdx_53e1ee5267ed
colorLab = [color: string, colorInd: double]
ohe = oneHot_48b48c72bfc3


oneHot_48b48c72bfc3

In [28]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.Tokenizer
val tkn = new Tokenizer().setInputCol("Description").setOutputCol("DescOut")
val tokenized = tkn.transform(sales.select("Description"))
tokenized.show(false)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, , water, transfer, tattoos]      |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

tkn = tok_742fb4b72cd1
tokenized = [Description: string, DescOut: array<string>]


[Description: string, DescOut: array<string>]

In [29]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.RegexTokenizer
val rt = new RegexTokenizer()
  .setInputCol("Description")
  .setOutputCol("DescOut")
  .setPattern(" ") // simplest expression
  .setToLowercase(true)
rt.transform(sales.select("Description")).show(false)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, water, transfer, tattoos]        |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

rt = regexTok_3af1d614c34c


regexTok_3af1d614c34c

In [30]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.RegexTokenizer
val rt = new RegexTokenizer()
  .setInputCol("Description")
  .setOutputCol("DescOut")
  .setPattern(" ")
  .setGaps(false)
  .setToLowercase(true)
rt.transform(sales.select("Description")).show(false)

+-----------------------------------+------------------+
|Description                        |DescOut           |
+-----------------------------------+------------------+
|RABBIT NIGHT LIGHT                 |[ ,  ]            |
|DOUGHNUT LIP GLOSS                 |[ ,  ,  ]         |
|12 MESSAGE CARDS WITH ENVELOPES    |[ ,  ,  ,  ]      |
|BLUE HARMONICA IN BOX              |[ ,  ,  ,  ]      |
|GUMBALL COAT RACK                  |[ ,  ]            |
|SKULLS  WATER TRANSFER TATTOOS     |[ ,  ,  ,  ,  ]   |
|FELTCRAFT GIRL AMELIE KIT          |[ ,  ,  ]         |
|CAMOUFLAGE LED TORCH               |[ ,  ]            |
|WHITE SKULL HOT WATER BOTTLE       |[ ,  ,  ,  ,  ]   |
|ENGLISH ROSE HOT WATER BOTTLE      |[ ,  ,  ,  ]      |
|HOT WATER BOTTLE KEEP CALM         |[ ,  ,  ,  ]      |
|SCOTTIE DOG HOT WATER BOTTLE       |[ ,  ,  ,  ]      |
|ROSE CARAVAN DOORSTOP              |[ ,  ]            |
|GINGHAM HEART  DOORSTOP RED        |[ ,  ,  ,  ]      |
|STORAGE TIN VINTAGE LEAF      

rt = regexTok_e8cc4d6e75e6


regexTok_e8cc4d6e75e6

In [31]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.StopWordsRemover
val englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
val stops = new StopWordsRemover()
  .setStopWords(englishStopWords)
  .setInputCol("DescOut")
stops.transform(tokenized).show()

+--------------------+--------------------+------------------------------+
|         Description|             DescOut|stopWords_5ce66be0f9f4__output|
+--------------------+--------------------+------------------------------+
|  RABBIT NIGHT LIGHT|[rabbit, night, l...|          [rabbit, night, l...|
| DOUGHNUT LIP GLOSS |[doughnut, lip, g...|          [doughnut, lip, g...|
|12 MESSAGE CARDS ...|[12, message, car...|          [12, message, car...|
|BLUE HARMONICA IN...|[blue, harmonica,...|          [blue, harmonica,...|
|   GUMBALL COAT RACK|[gumball, coat, r...|          [gumball, coat, r...|
|SKULLS  WATER TRA...|[skulls, , water,...|          [skulls, , water,...|
|FELTCRAFT GIRL AM...|[feltcraft, girl,...|          [feltcraft, girl,...|
|CAMOUFLAGE LED TORCH|[camouflage, led,...|          [camouflage, led,...|
|WHITE SKULL HOT W...|[white, skull, ho...|          [white, skull, ho...|
|ENGLISH ROSE HOT ...|[english, rose, h...|          [english, rose, h...|
|HOT WATER BOTTLE ...|[ho

englishStopWords = Array(i, me, my, myself, we, our, ours, ourselves, you, your, yours, yourself, yourselves, he, him, his, himself, she, her, hers, herself, it, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, ver...


[i, me, my, myself, we, our, ours, ourselves, you, your, yours, yourself, yourselves, he, him, his, himself, she, her, hers, herself, it, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, should, now, i'll, you'll, he'll, she'll, we'll, they'll, i'd, you'd, he'd, she'd, we'd, they'd, i'm, you're, he's, she's, it's, we're, they're, i've, we've, you've, they've, isn't, aren't, wasn't, weren't, haven't, hasn't, hadn't, don't, doesn't, didn't, won't, 

In [32]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.NGram
val unigram = new NGram().setInputCol("DescOut").setN(1)
val bigram = new NGram().setInputCol("DescOut").setN(2)
unigram.transform(tokenized.select("DescOut")).show(false)
bigram.transform(tokenized.select("DescOut")).show(false)

+------------------------------------------+------------------------------------------+
|DescOut                                   |ngram_d598d1ea91f8__output                |
+------------------------------------------+------------------------------------------+
|[rabbit, night, light]                    |[rabbit, night, light]                    |
|[doughnut, lip, gloss]                    |[doughnut, lip, gloss]                    |
|[12, message, cards, with, envelopes]     |[12, message, cards, with, envelopes]     |
|[blue, harmonica, in, box]                |[blue, harmonica, in, box]                |
|[gumball, coat, rack]                     |[gumball, coat, rack]                     |
|[skulls, , water, transfer, tattoos]      |[skulls, , water, transfer, tattoos]      |
|[feltcraft, girl, amelie, kit]            |[feltcraft, girl, amelie, kit]            |
|[camouflage, led, torch]                  |[camouflage, led, torch]                  |
|[white, skull, hot, water, bott

unigram = ngram_d598d1ea91f8
bigram = ngram_e23490dc63d7


ngram_e23490dc63d7

In [33]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.CountVectorizer
val cv = new CountVectorizer()
  .setInputCol("DescOut")
  .setOutputCol("countVec")
  .setVocabSize(500)
  .setMinTF(1)
  .setMinDF(2)
val fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show(false)

+-----------------------------------+------------------------------------------+---------------------------------------------------+
|Description                        |DescOut                                   |countVec                                           |
+-----------------------------------+------------------------------------------+---------------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |(500,[149,185,212],[1.0,1.0,1.0])                  |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |(500,[462,463,492],[1.0,1.0,1.0])                  |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |(500,[35,41,166],[1.0,1.0,1.0])                    |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |(500,[10,16,36,352],[1.0,1.0,1.0,1.0])             |
|GUMBALL COAT RACK                  |[gumball, coat, rack]           

cv = cntVec_cc5b21a018ef
fittedCV = cntVec_cc5b21a018ef


cntVec_cc5b21a018ef

In [34]:
// COMMAND ----------

// in Scala
val tfIdfIn = tokenized
  .where("array_contains(DescOut, 'red')")
  .select("DescOut")
  .limit(10)
tfIdfIn.show(false)

+--------------------------------------+
|DescOut                               |
+--------------------------------------+
|[wooden, advent, calendar, red]       |
|[wrap, red, vintage, doily]           |
|[alarm, clock, bakelike, red]         |
|[alarm, clock, bakelike, red]         |
|[red, diner, wall, clock]             |
|[red, apples, chopping, board]        |
|[red, kitchen, scales]                |
|[airline, bag, vintage, jet, set, red]|
|[red, retrospot, charlotte, bag]      |
|[pin, cushion, babushka, red]         |
+--------------------------------------+



tfIdfIn = [DescOut: array<string>]


[DescOut: array<string>]

In [35]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.{HashingTF, IDF}
val tf = new HashingTF()
  .setInputCol("DescOut")
  .setOutputCol("TFOut")
  .setNumFeatures(10000)
val idf = new IDF()
  .setInputCol("TFOut")
  .setOutputCol("IDFOut")
  .setMinDocFreq(2)

tf = hashingTF_beae0d33235e
idf = idf_b0d3a2391ddf


idf_b0d3a2391ddf

In [36]:
// COMMAND ----------

// in Scala
idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).show(false)

+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------+
|DescOut                                |TFOut                                                   |IDFOut                                                        |
+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------+
|[red, hanging, heart, t-light, holder] |(10000,[538,3348,4291,6152,9160],[1.0,1.0,1.0,1.0,1.0]) |(10000,[538,3348,4291,6152,9160],[0.0,0.0,0.0,0.0,0.0])       |
|[set/10, red, polkadot, party, candles]|(10000,[3686,4291,5092,9195,9369],[1.0,1.0,1.0,1.0,1.0])|(10000,[3686,4291,5092,9195,9369],[0.0,0.0,0.0,0.0,0.0])      |
|[jumbo, bag, red, retrospot]           |(10000,[155,1174,2591,4291],[1.0,1.0,1.0,1.0])          |(10000,[155,1174,2591,4291],[1.2992829841302609,0.0,0.0,0.0]) |
|[vintage, red, kitchen, cab

In [37]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.Word2Vec
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
// Input data: Each row is a bag of words from a sentence or document.
val documentDF = spark.createDataFrame(Seq(
  "Hi I heard about Spark".split(" "),
  "I wish Java could use case classes".split(" "),
  "Logistic regression models are neat".split(" ")
).map(Tuple1.apply)).toDF("text")

documentDF = [text: array<string>]


[text: array<string>]

In [38]:
// Learn a mapping from words to Vectors.
val word2Vec = new Word2Vec()
  .setInputCol("text")
  .setOutputCol("result")
  .setVectorSize(3)
  .setMinCount(0)

word2Vec = w2v_907d03a2d51b


w2v_907d03a2d51b

In [39]:
val model = word2Vec.fit(documentDF)
val result = model.transform(documentDF)

model = w2v_907d03a2d51b
result = [text: array<string>, result: vector]


[text: array<string>, result: vector]

In [40]:
result.collect().foreach { case Row(text: Seq[_], features: Vector) =>
  println(s"Text: [${text.mkString(", ")}] => \nVector: $features\n")
}

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.008301425352692604,0.020340083539485933,0.03263562321662903]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.04305013721542699,0.035192844324878285,0.02361524730388607]

Text: [Logistic, regression, models, are, neat] => 
Vector: [0.0386008620262146,-0.032484661601483826,-0.015685077011585235]



In [41]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.PCA
val pca = new PCA().setInputCol("features").setK(2)
pca.fit(scaleDF).transform(scaleDF).show(false)

+---+--------------+------------------------------------------+
|id |features      |pca_6948161da487__output                  |
+---+--------------+------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
|1  |[2.0,1.1,1.0] |[-1.6804946984073725,1.2593401322219144]  |
|0  |[1.0,0.1,-1.0]|[0.07137194992484153,-0.45266548881478463]|
|1  |[2.0,1.1,1.0] |[-1.6804946984073725,1.2593401322219144]  |
|1  |[3.0,10.1,3.0]|[-10.872398139848944,0.030962697060149758]|
+---+--------------+------------------------------------------+



pca = pca_6948161da487


pca_6948161da487

In [42]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.PolynomialExpansion
val pe = new PolynomialExpansion().setInputCol("features").setDegree(2)
pe.transform(scaleDF).show(false)

+---+--------------+-----------------------------------------------------------------------------------+
|id |features      |poly_d59f6f8e9e1b__output                                                          |
+---+--------------+-----------------------------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[1.0,1.0,0.1,0.1,0.010000000000000002,-1.0,-1.0,-0.1,1.0]                          |
|1  |[2.0,1.1,1.0] |[2.0,4.0,1.1,2.2,1.2100000000000002,1.0,2.0,1.1,1.0]                               |
|0  |[1.0,0.1,-1.0]|[1.0,1.0,0.1,0.1,0.010000000000000002,-1.0,-1.0,-0.1,1.0]                          |
|1  |[2.0,1.1,1.0] |[2.0,4.0,1.1,2.2,1.2100000000000002,1.0,2.0,1.1,1.0]                               |
|1  |[3.0,10.1,3.0]|[3.0,9.0,10.1,30.299999999999997,102.00999999999999,3.0,9.0,30.299999999999997,9.0]|
+---+--------------+-----------------------------------------------------------------------------------+



pe = poly_d59f6f8e9e1b


poly_d59f6f8e9e1b

In [43]:
sales.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [44]:
// COMMAND ----------

// in Scala
// import org.apache.spark.ml.feature.{ChiSqSelector, Tokenizer}
// val tkn = new Tokenizer().setInputCol("Description").setOutputCol("DescOut")

// val tokenized = tkn
//   .transform(sales.select("Description", "CustomerID"))
//   .where("CustomerID IS NOT NULL")

// tokenized.show()
//val prechi = fittedCV.transform(tokenized)

//// val prechi = fittedCV.transform(tokenized).join(sales.select("Description", "CustomerID"),"Description")
//// .where("CustomerID IS NOT NULL")

// prechi.show()

// val chisq = new ChiSqSelector().setFeaturesCol("countVec").setLabelCol("CustomerID").setNumTopFeatures(2)

// chisq.fit(prechi).transform(prechi)
//   .drop("CustomerID", "Description", "DescOut").show()

//CHECK THIS TAKING TOO LONG ...AND CHECK CODE for JOIN which I inserted!!!

Name: Syntax Error.
Message: 
StackTrace: 

In [45]:
// COMMAND ----------

// in Scala
val fittedPCA = pca.fit(scaleDF)
fittedPCA.write.overwrite().save("/tmp/fittedPCA")

fittedPCA = pca_6948161da487


pca_6948161da487

In [46]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.PCAModel
val loadedPCA = PCAModel.load("/tmp/fittedPCA")
loadedPCA.transform(scaleDF).show()

+---+--------------+------------------------+
| id|      features|pca_6948161da487__output|
+---+--------------+------------------------+
|  0|[1.0,0.1,-1.0]|    [0.07137194992484...|
|  1| [2.0,1.1,1.0]|    [-1.6804946984073...|
|  0|[1.0,0.1,-1.0]|    [0.07137194992484...|
|  1| [2.0,1.1,1.0]|    [-1.6804946984073...|
|  1|[3.0,10.1,3.0]|    [-10.872398139848...|
+---+--------------+------------------------+



loadedPCA = pca_6948161da487


pca_6948161da487

In [47]:
import org.apache.spark.ml.UnaryTransformer
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.types.{ArrayType, StringType, DataType}
import org.apache.spark.ml.param.{IntParam, ParamValidators}

class MyTokenizer(override val uid: String) 
    extends UnaryTransformer[String, Seq[String], MyTokenizer] 
    with DefaultParamsWritable {

  def this() = this(Identifiable.randomUID("myTokenizer"))

  val maxWords: IntParam = new IntParam(this, 
                                        "maxWords",
                                        "The max number of words to return.",
                                        ParamValidators.gtEq(0))

  def setMaxWords(value: Int): this.type = set(maxWords, value)

  def getMaxWords: Integer = $(maxWords)

  override protected def createTransformFunc: String => Seq[String] = (
    inputString: String) => {
      inputString.split("\\s").take($(maxWords))
  }

  override protected def validateInputType(inputType: DataType): Unit = {
    require(
      inputType == StringType, s"Bad input type: $inputType. Requires String.")
  }

  override protected def outputDataType: DataType = new ArrayType(StringType,
    true)
}
// this will allow you to read it back in by using this object.
object MyTokenizer extends DefaultParamsReadable[MyTokenizer]

defined class MyTokenizer
defined object MyTokenizer


In [48]:
// COMMAND ----------

val myT = new MyTokenizer().setInputCol("someCol").setMaxWords(2)

myT.transform(Seq("hello world. This text won't show.").toDF("someCol")).show(3, false)


// COMMAND ----------

+----------------------------------+--------------------------------+
|someCol                           |myTokenizer_82829566c98d__output|
+----------------------------------+--------------------------------+
|hello world. This text won't show.|[hello, world.]                 |
+----------------------------------+--------------------------------+



myT = myTokenizer_82829566c98d


myTokenizer_82829566c98d

In [49]:
val myT1 = new MyTokenizer().setInputCol("someCol").setMaxWords(3)

myT1.transform(Seq("hello world. This text won't show.").toDF("someCol")).show(3, false)


+----------------------------------+--------------------------------+
|someCol                           |myTokenizer_72ed2fed1f48__output|
+----------------------------------+--------------------------------+
|hello world. This text won't show.|[hello, world., This]           |
+----------------------------------+--------------------------------+



myT1 = myTokenizer_72ed2fed1f48


myTokenizer_72ed2fed1f48